In [4]:
import numpy as np
from data import load_dataset

## Read data

In [6]:
train_x, train_y, test_x, test_y, classes = load_dataset()
print("number of samples in train dataset: ", train_x.shape[0])
print("number of samples in test  dataset: ", test_x.shape[0])
print("shape of train_x: ", train_x.shape)
print("shape of train_y: ", train_y.shape)
print("shape of test_x: ", test_x.shape)
print("shape of test_y: ", test_y.shape)

number of samples in train dataset:  209
number of samples in test  dataset:  50
shape of train_x:  (209, 64, 64, 3)
shape of train_y:  (1, 209)
shape of test_x:  (50, 64, 64, 3)
shape of test_y:  (1, 50)


In [7]:
train_y

array([[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [21]:
train_x = train_x.reshape(train_x.shape[0], -1)
train_y = train_y.reshape(train_y.shape[1], -1)
test_x = test_x.reshape(test_x.shape[0], -1)
test_y = test_y.reshape(test_y.shape[1], -1)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(209, 12288) (209, 1) (50, 12288) (50, 1)


## Data standardization processing

In [10]:
train_x = train_x / 255.
test_x = test_x / 255.

## sigmoid function

In [11]:
def basic_sigmoid(x):
    return 1 / (1 + np.exp(-x))

## Initialize parameters

In [78]:
def init_params(num_features): 
    """
    Create a w parameter with a shape of (num_features, 1) and b=0
    return:w, b
    """
    w = np.zeros((num_features, 1))
    b = 0.0
    
    return w, b


# test init_with_params function
w, b = init_params(4)
print(w.shape)
print(b)
    

(4, 1)
0.0


## Forward and backward propagation
compute dw,db and cost

In [65]:
def propagate(w, b, X, Y):
    """
    Params:
        w: weights, shape: (num_features, 1)
        b: bias
        X: x data, shape: (m, num_features)
        Y: labels, shape:(m, 1)
    Return:
        cost、dw、db
    """
    m = X.shape[0]

    # forward
    #  X: (m, num_features), w:(num_features, 1),, z: (m, 1)
    z = (np.dot(X, w) + b).reshape(m, 1)
    # A shape: (m, 1)
    A = basic_sigmoid(z)
    
    #cost
    cost = (-1 / m) * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))

    # backward
    # dz.shape: (m, 1)
    dz = A - Y
    # dw.shape: (num_features, 1), X.T: (num_features, m), dz: (m, 1)
    dw = (1 / m) * np.dot(X.T, dz)
    db = (1 / m) * np.sum(dz)

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())

    grads = {"dw": dw, "db": db}

    return grads, cost

In [66]:
# test propagate function
num_features = train_x.shape[1]
w, b = init_with_params(num_features)

grads, cost = propagate(w, b, train_x, train_y)
print(grads["dw"].shape)
print(grads["dw"])
print(grads["db"])
print(cost)

(12288, 1)
[[0.04720893]
 [0.06299841]
 [0.04923539]
 ...
 [0.05074585]
 [0.0621259 ]
 [0.03245145]]
0.15550239234449761
0.6931471805599453


## optimizing process
update w and b, and save cost

In [70]:
def optimize(w, b, X, Y, num_iterations, learning_rate):
    """
    params：
        w: weights, shape: (num_features, 1)
        b: bias
        X: x data, shape: (m, num_features)
        Y: labels, shape:(m, 1)
        num_iterations: number of iterations
        learning_rate: learning rate
    Returns:
        params: {"w": w, "b": b}
        costs: cost values, [float]
    """
    costs = []
    for i in range(num_iterations):

        # get grads and cost
        grads, cost = propagate(w, b, X, Y)

        # get dw, db
        dw = grads["dw"]
        db = grads["db"]

        # update w and b
        w = w - learning_rate * dw
        b = b - learning_rate * db

        # save cost
        if i % 100 == 0:
            costs.append(cost)
            print("cost result %i: %f" %(i, cost))

    params = {"w": w, "b": b}

    return params, costs
        

In [69]:
# test optimize function
optimize(w, b, train_x, train_y, 201, 0.01)

cost result 0: 0.693147
cost result 100: 0.823921
cost result 200: 0.418944


({'w': array([[ 0.00664283],
         [-0.01163286],
         [-0.00548023],
         ...,
         [-0.00842764],
         [-0.01770452],
         [ 0.00787242]]),
  'b': -0.003863357901216874},
 [0.6931471805599453, 0.8239208681632532, 0.41894444797388297])

## predict

In [74]:
def precit(w, b, X):
    """
    params：
        w: weights, shape: (num_features, 1)
        b: bias
        X: x data, shape: (m, num_features)
    Returns:
        Y_prediction: shape: (m, 1) 
    """
    m = X.shape[0]

    # forward
    #  X: (m, num_features), w:(num_features, 1),, z: (m, 1)
    z = (np.dot(X, w) + b).reshape(m, 1)
    # A shape: (m, 1)
    A = basic_sigmoid(z)

    Y_prediction = np.where(A > 0.5, 1, 0)

    assert(Y_prediction.shape == (m, 1))

    return Y_prediction
    

In [76]:
# test predict function
Y_prediction = precit(w, b, train_x)
print(Y_prediction.shape)

(209, 1)


## implement all 

In [87]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.01):
    num_features = X_train.shape[1]
    
    # init params
    w, b = init_params(num_features)

    # train params
    params, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate)

    # get w and b
    w = params["w"]
    b = params["b"]

    # predict
    Y_prediction_train = precit(w, b, X_train)
    Y_prediction_test = precit(w, b, X_test)

    # print accuracy
    print("train data accuracy: {}".format(np.mean(Y_prediction_train == Y_train)))
    print("test  data accuracy: {}".format(np.mean(Y_prediction_test == Y_test)))

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

## test

In [88]:
d = model(train_x, train_y, test_x, test_y, num_iterations = 2000, learning_rate = 0.005)

cost result 0: 0.693147
cost result 100: 0.584508
cost result 200: 0.466949
cost result 300: 0.376007
cost result 400: 0.331463
cost result 500: 0.303273
cost result 600: 0.279880
cost result 700: 0.260042
cost result 800: 0.242941
cost result 900: 0.228004
cost result 1000: 0.214820
cost result 1100: 0.203078
cost result 1200: 0.192544
cost result 1300: 0.183033
cost result 1400: 0.174399
cost result 1500: 0.166521
cost result 1600: 0.159305
cost result 1700: 0.152667
cost result 1800: 0.146542
cost result 1900: 0.140872
train data accuracy: 0.9904306220095693
test  data accuracy: 0.7
